In [5]:
import pandas as pd
import time
import numpy as np
import sqlite3
import json

import IPython
from IPython.display import display, SVG, clear_output, HTML
from ipywidgets import widgets
import ipywidgets as widgets
from IPython.display import display, clear_output

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import seaborn as sns

import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px
import plotly.graph_objs as go
from sklearn.linear_model import LinearRegression

import numpy as np

In [6]:
con = sqlite3.connect('prices.db')
stocks = pd.read_sql("SELECT * FROM stocks;", con)
amazon_items = pd.read_sql("SELECT * FROM amazon_items;", con)
cryptocurrencies = pd.read_sql("SELECT * FROM cryptocurrencies;", con)
DATA_FILE = 'trackitems.json'
con.close()

In [7]:
# Define the path to your JSON file
DATA_FILE = 'trackitems.json'

# Function to read data from JSON file
def read_data(filename):
    try:
        with open(filename, 'r') as file:
            return json.load(file)
    except FileNotFoundError:
        return {"amazon_items": [], "stock_tickers": [], "crypto_names": "", "portfolio": []}

# Function to write data to JSON file
def write_data(filename, data):
    with open(filename, 'w') as file:
        json.dump(data, file, indent=4)

# Read data from file
data = read_data(DATA_FILE)
amazon_item_list = data['amazon_items']
stock_tickers = data['stocks']
crypto_names = data['cryptocurrencies']

# Display current lists
amazon_label = widgets.HTML(value="<b>Current Amazon Items:</b>")
amazon_current = widgets.Textarea(value='\n'.join(amazon_item_list), disabled=True, layout=widgets.Layout(width='100%', height='100px'))

stocks_label = widgets.HTML(value="<b>Current Stocks:</b>")
stocks_current = widgets.Textarea(value='\n'.join(stock_tickers), disabled=True, layout=widgets.Layout(width='100%', height='100px'))

cryptos_label = widgets.HTML(value="<b>Current Cryptocurrencies:</b>")
cryptos_current = widgets.Textarea(value=crypto_names, disabled=True, layout=widgets.Layout(width='100%', height='100px'))

# Create text areas for editing
amazon_edit = widgets.Textarea(value='\n'.join(amazon_item_list), placeholder='Add Amazon items', description='Edit:', layout=widgets.Layout(width='100%', height='150px'))
stocks_edit = widgets.Textarea(value='\n'.join(stock_tickers), placeholder='Add stocks', description='Edit:', layout=widgets.Layout(width='100%', height='150px'))
cryptos_edit = widgets.Textarea(value=crypto_names, placeholder='Add cryptocurrencies separated by commas', description='Edit:', layout=widgets.Layout(width='100%', height='150px'))

# Create update button
update_button = widgets.Button(description='Update Lists', button_style='success', layout=widgets.Layout(width='30%', height='40px'))

# Output area for messages
output = widgets.Output()

# Function to update lists
def update_lists(b):
    data['amazon_items'] = amazon_edit.value.split('\n')
    data['stock_tickers'] = stocks_edit.value.split('\n')
    data['crypto_names'] = cryptos_edit.value
    
    write_data(DATA_FILE, data)
    
    # Update the current displays
    amazon_current.value = '\n'.join(data['amazon_items'])
    stocks_current.value = '\n'.join(data['stock_tickers'])
    cryptos_current.value = data['crypto_names']
    
    with output:
        clear_output()
        print('Lists updated successfully!')

# Attach update function to button
update_button.on_click(update_lists)

# Organize widgets in tabs
tab = widgets.Tab()
tab.children = [
    widgets.VBox([amazon_label, amazon_current, amazon_edit, update_button]), 
    widgets.VBox([stocks_label, stocks_current, stocks_edit, update_button]),
    widgets.VBox([cryptos_label, cryptos_current, cryptos_edit, update_button])
]
tab.set_title(0, 'Amazon Items')
tab.set_title(1, 'Stocks')
tab.set_title(2, 'Cryptocurrencies')

# Display all widgets in a vertical box
ui = widgets.VBox([tab, output])
display(ui)


In [8]:

# Convert 'timestamp' column to datetime if not already in datetime format
if not pd.api.types.is_datetime64_any_dtype(stocks['timestamp']):
    stocks['timestamp'] = pd.to_datetime(stocks['timestamp'])

if not pd.api.types.is_datetime64_any_dtype(cryptocurrencies['timestamp']):
    cryptocurrencies['timestamp'] = pd.to_datetime(cryptocurrencies['timestamp'])

# Sort the data by stock_symbol/crypto_name and timestamp
stocks = stocks.sort_values(by=['stock_symbol', 'timestamp'])
cryptocurrencies = cryptocurrencies.sort_values(by=['crypto_name', 'timestamp'])

# Calculate moving averages, Bollinger Bands, daily returns, and cumulative returns for stocks
def calculate_indicators(df, group_column):
    df['20_MA'] = df.groupby(group_column)['price'].transform(lambda x: x.rolling(window=20, min_periods=1).mean())
    df['20_STD'] = df.groupby(group_column)['price'].transform(lambda x: x.rolling(window=20, min_periods=1).std())
    df['Upper_BB'] = df['20_MA'] + (df['20_STD'] * 2)
    df['Lower_BB'] = df['20_MA'] - (df['20_STD'] * 2)
    df['daily_return'] = df.groupby(group_column)['price'].pct_change()
    df['cumulative_return'] = (1 + df['daily_return']).cumprod()
    return df

stocks = calculate_indicators(stocks, 'stock_symbol')
cryptocurrencies = calculate_indicators(cryptocurrencies, 'crypto_name')

# Read portfolio from JSON file
DATA_FILE = 'trackitems.json'
def read_data(filename):
    try:
        with open(filename, 'r') as file:
            return json.load(file)
    except FileNotFoundError:
        return {"amazon_items": [], "stock_tickers": [], "crypto_names": "", "portfolio": [], "crypto_portfolio": []}

def write_data(filename, data):
    with open(filename, 'w') as file:
        json.dump(data, file, indent=4)

data = read_data(DATA_FILE)
portfolio = data['portfolio']
crypto_portfolio = data['crypto_portfolio']

# Create the Dash app
app = dash.Dash(__name__)

# Layout of the Dash app
app.layout = html.Div([
    html.H1("Investment Dashboard"),
    dcc.Tabs([
        dcc.Tab(label='Stock Analysis', children=[
            dcc.Dropdown(
                id='stock-dropdown',
                options=[{'label': stock, 'value': stock} for stock in stocks['stock_symbol'].unique()],
                value=stocks['stock_symbol'].unique()[0]
            ),
            dcc.Checklist(
                id='toggle-options',
                options=[
                    {'label': 'Show Moving Average', 'value': 'show_ma'},
                    {'label': 'Show Bollinger Bands', 'value': 'show_bb'},
                    {'label': 'Show Trend Line', 'value': 'show_trend'}
                ],
                value=['show_ma', 'show_bb', 'show_trend'],
                labelStyle={'display': 'inline-block'}
            ),
            dcc.Graph(id='price-graph'),
            html.Div([
                dcc.DatePickerSingle(
                    id='investment-date-picker',
                    min_date_allowed=stocks['timestamp'].min().date(),
                    max_date_allowed=stocks['timestamp'].max().date(),
                    initial_visible_month=stocks['timestamp'].min().date(),
                    date=stocks['timestamp'].min().date()
                ),
                html.Div(id='investment-value')
            ]),
            html.Div([
                dcc.DatePickerSingle(
                    id='lowest-price-date-picker',
                    min_date_allowed=stocks['timestamp'].min().date(),
                    max_date_allowed=stocks['timestamp'].max().date(),
                    initial_visible_month=stocks['timestamp'].min().date(),
                    date=stocks['timestamp'].min().date()
                ),
                html.Div(id='lowest-price')
            ]),
            dcc.Graph(id='correlation-matrix'),
            dcc.Graph(id='daily-returns-graph'),
            dcc.Graph(id='cumulative-returns-graph')
        ]),
        dcc.Tab(label='Crypto Analysis', children=[
            dcc.Dropdown(
                id='crypto-dropdown',
                options=[{'label': crypto, 'value': crypto} for crypto in cryptocurrencies['crypto_name'].unique()],
                value=cryptocurrencies['crypto_name'].unique()[0]
            ),
            dcc.Checklist(
                id='crypto-toggle-options',
                options=[
                    {'label': 'Show Moving Average', 'value': 'show_ma'},
                    {'label': 'Show Bollinger Bands', 'value': 'show_bb'},
                    {'label': 'Show Trend Line', 'value': 'show_trend'}
                ],
                value=['show_ma', 'show_bb', 'show_trend'],
                labelStyle={'display': 'inline-block'}
            ),
            dcc.Graph(id='crypto-price-graph'),
            html.Div([
                dcc.DatePickerSingle(
                    id='crypto-investment-date-picker',
                    min_date_allowed=cryptocurrencies['timestamp'].min().date(),
                    max_date_allowed=cryptocurrencies['timestamp'].max().date(),
                    initial_visible_month=cryptocurrencies['timestamp'].min().date(),
                    date=cryptocurrencies['timestamp'].min().date()
                ),
                html.Div(id='crypto-investment-value')
            ]),
            html.Div([
                dcc.DatePickerSingle(
                    id='crypto-lowest-price-date-picker',
                    min_date_allowed=cryptocurrencies['timestamp'].min().date(),
                    max_date_allowed=cryptocurrencies['timestamp'].max().date(),
                    initial_visible_month=cryptocurrencies['timestamp'].min().date(),
                    date=cryptocurrencies['timestamp'].min().date()
                ),
                html.Div(id='crypto-lowest-price')
            ]),
            dcc.Graph(id='crypto-correlation-matrix'),
            dcc.Graph(id='crypto-daily-returns-graph'),
            dcc.Graph(id='crypto-cumulative-returns-graph')
        ]),
        dcc.Tab(label='Portfolio Management', children=[
            html.H3("Manage Portfolio"),
            dcc.Input(
                id='portfolio-stock-input',
                type='text',
                placeholder='Add stock to portfolio',
                debounce=True
            ),
            html.Div([
                dcc.DatePickerSingle(
                    id='portfolio-date-picker',
                    min_date_allowed=stocks['timestamp'].min().date(),
                    max_date_allowed=stocks['timestamp'].max().date(),
                    initial_visible_month=stocks['timestamp'].min().date()
                ),
                dcc.Input(
                    id='portfolio-quantity-input',
                    type='number',
                    placeholder='Quantity',
                    min=1,
                ),
                dcc.Input(
                    id='portfolio-price-input',
                    type='number',
                    placeholder='Price',
                    min=0.01,
                    step=0.01,
                )
            ]),
            html.Button('Add to Portfolio', id='add-to-portfolio-button', n_clicks=0),
            html.Button('Remove Selected Stock', id='remove-from-portfolio-button', n_clicks=0),
            dcc.Dropdown(
                id='portfolio-remove-dropdown',
                options=[{'label': stock['symbol'], 'value': stock['symbol']} for stock in portfolio],
                placeholder='Select stock to remove'
            ),
            html.H3("Portfolio"),
            html.Div(id='portfolio-display'),
            html.H3("Portfolio Metrics"),
            html.Div(id='portfolio-metrics'),
            dcc.Graph(id='stock-distribution'),
            html.H3("Manage Crypto Portfolio"),
            dcc.Input(
                id='portfolio-crypto-input',
                type='text',
                placeholder='Add crypto to portfolio',
                debounce=True
            ),
            html.Div([
                dcc.DatePickerSingle(
                    id='crypto-portfolio-date-picker',
                    min_date_allowed=cryptocurrencies['timestamp'].min().date(),
                    max_date_allowed=cryptocurrencies['timestamp'].max().date(),
                    initial_visible_month=cryptocurrencies['timestamp'].min().date()
                ),
                dcc.Input(
                    id='crypto-portfolio-quantity-input',
                    type='number',
                    placeholder='Quantity',
                    min=1,
                ),
                dcc.Input(
                    id='crypto-portfolio-price-input',
                    type='number',
                    placeholder='Price',
                    min=0.01,
                    step=0.01,
                )
            ]),
            html.Button('Add to Crypto Portfolio', id='add-to-crypto-portfolio-button', n_clicks=0),
            html.Button('Remove Selected Crypto', id='remove-from-crypto-portfolio-button', n_clicks=0),
            dcc.Dropdown(
                id='crypto-portfolio-remove-dropdown',
                options=[{'label': crypto['symbol'], 'value': crypto['symbol']} for crypto in crypto_portfolio],
                placeholder='Select crypto to remove'
            ),
            html.H3("Crypto Portfolio"),
            html.Div(id='crypto-portfolio-display'),
            html.H3("Crypto Portfolio Metrics"),
            html.Div(id='crypto-portfolio-metrics'),
            dcc.Graph(id='crypto-distribution')
        ])
    ])
])

# Common callback functions for stocks and cryptocurrencies

def update_graph(df, selected_item, toggle_options, group_column, ma_column, upper_bb_column, lower_bb_column):
    item_data = df[df[group_column] == selected_item]
    
    traces = []

    price_trace = go.Scatter(
        x=item_data['timestamp'],
        y=item_data['price'],
        mode='lines+markers',
        name='Price'
    )
    traces.append(price_trace)
    
    if 'show_ma' in toggle_options:
        ma_trace = go.Scatter(
            x=item_data['timestamp'],
            y=item_data[ma_column],
            mode='lines',
            name='20-Day MA',
            line=dict(width=1)
        )
        traces.append(ma_trace)
    
    if 'show_bb' in toggle_options:
        upper_bb_trace = go.Scatter(
            x=item_data['timestamp'],
            y=item_data[upper_bb_column],
            mode='lines',
            name='Upper Bollinger Band',
            line=dict(color='rgba(0,100,80,0.2)', width=1),
        )
        lower_bb_trace = go.Scatter(
            x=item_data['timestamp'],
            y=item_data[lower_bb_column],
            mode='lines',
            name='Lower Bollinger Band',
            line=dict(color='rgba(0,100,80,0.2)', width=1),
            fill='tonexty'
        )
        traces.extend([upper_bb_trace, lower_bb_trace])
    
    if 'show_trend' in toggle_options:
        x_values = np.arange(len(item_data))
        y_values = item_data['price'].values
        if len(x_values) > 1:
            with np.warnings.catch_warnings():
                np.warnings.simplefilter('ignore', np.RankWarning)
                coeffs = np.polyfit(x_values, y_values, 1)
                polynomial = np.poly1d(coeffs)
                trendline = polynomial(x_values)
            
            trend_trace = go.Scatter(
                x=item_data['timestamp'],
                y=trendline,
                mode='lines',
                name='Trend Line',
                line=dict(width=1)
            )
            traces.append(trend_trace)
    
    layout = go.Layout(
        title=f'Prices Over Time for {selected_item}',
        xaxis={'title': 'Timestamp'},
        yaxis={'title': 'Price'},
        legend={'x': 0, 'y': 1},
        hovermode='closest'
    )
    
    return {'data': traces, 'layout': layout}

# Callback for stock graph
@app.callback(
    Output('price-graph', 'figure'),
    [Input('stock-dropdown', 'value'),
     Input('toggle-options', 'value')]
)
def update_stock_graph(selected_stock, toggle_options):
    return update_graph(stocks, selected_stock, toggle_options, 'stock_symbol', '20_MA', 'Upper_BB', 'Lower_BB')

# Callback for crypto graph
@app.callback(
    Output('crypto-price-graph', 'figure'),
    [Input('crypto-dropdown', 'value'),
     Input('crypto-toggle-options', 'value')]
)
def update_crypto_graph(selected_crypto, toggle_options):
    return update_graph(cryptocurrencies, selected_crypto, toggle_options, 'crypto_name', '20_MA', 'Upper_BB', 'Lower_BB')

# Callback for investment value
@app.callback(
    Output('investment-value', 'children'),
    [Input('stock-dropdown', 'value'),
     Input('investment-date-picker', 'date')]
)
def update_investment_value(selected_stock, investment_date):
    investment_date = pd.to_datetime(investment_date)
    stock_data = stocks[(stocks['stock_symbol'] == selected_stock) & (stocks['timestamp'].dt.date == investment_date.date())]
    
    if stock_data.empty:
        return "No data available for the selected date."

    initial_price = stock_data.iloc[0]['price']
    current_price = stocks[stocks['stock_symbol'] == selected_stock].iloc[-1]['price']
    
    investment_value = 1000 * (current_price / initial_price)
    
    return f'The current value of a $1000 investment in {selected_stock} on {investment_date.date()} is ${investment_value:.2f}.'

# Callback for crypto investment value
@app.callback(
    Output('crypto-investment-value', 'children'),
    [Input('crypto-dropdown', 'value'),
     Input('crypto-investment-date-picker', 'date')]
)
def update_crypto_investment_value(selected_crypto, investment_date):
    investment_date = pd.to_datetime(investment_date)
    crypto_data = cryptocurrencies[(cryptocurrencies['crypto_name'] == selected_crypto) & (cryptocurrencies['timestamp'].dt.date == investment_date.date())]
    
    if crypto_data.empty:
        return "No data available for the selected date."

    initial_price = crypto_data.iloc[0]['price']
    current_price = cryptocurrencies[cryptocurrencies['crypto_name'] == selected_crypto].iloc[-1]['price']
    
    investment_value = 1000 * (current_price / initial_price)
    
    return f'The current value of a $1000 investment in {selected_crypto} on {investment_date.date()} is ${investment_value:.2f}.'

# Callback to display the lowest price since a given date for stocks
@app.callback(
    Output('lowest-price', 'children'),
    [Input('stock-dropdown', 'value'),
     Input('lowest-price-date-picker', 'date')]
)
def update_lowest_price(selected_stock, since_date):
    since_date = pd.to_datetime(since_date)
    stock_data = stocks[(stocks['stock_symbol'] == selected_stock) & (stocks['timestamp'] >= since_date)]
    
    if stock_data.empty:
        return f"No data available for {selected_stock} since {since_date.date()}."

    lowest_price = stock_data['price'].min()
    
    return f'The lowest recorded price for {selected_stock} since {since_date.date()} is ${lowest_price:.2f}.'

# Callback to display the lowest price since a given date for cryptocurrencies
@app.callback(
    Output('crypto-lowest-price', 'children'),
    [Input('crypto-dropdown', 'value'),
     Input('crypto-lowest-price-date-picker', 'date')]
)
def update_crypto_lowest_price(selected_crypto, since_date):
    since_date = pd.to_datetime(since_date)
    crypto_data = cryptocurrencies[(cryptocurrencies['crypto_name'] == selected_crypto) & (cryptocurrencies['timestamp'] >= since_date)]
    
    if crypto_data.empty:
        return f"No data available for {selected_crypto} since {since_date.date()}."

    lowest_price = crypto_data['price'].min()
    
    return f'The lowest recorded price for {selected_crypto} since {since_date.date()} is ${lowest_price:.2f}.'

# Callback to update the correlation matrix for stocks
@app.callback(
    Output('correlation-matrix', 'figure'),
    [Input('stock-dropdown', 'value')]
)
def update_correlation_matrix(selected_stock):
    pivot_df = stocks.pivot(index='timestamp', columns='stock_symbol', values='price')
    correlation_matrix = pivot_df.corr()

    fig = px.imshow(correlation_matrix,
                    x=correlation_matrix.columns,
                    y=correlation_matrix.columns,
                    color_continuous_scale='RdBu_r',
                    zmin=-1, zmax=1)
    fig.update_layout(title='Correlation Matrix of Stock Prices')
    
    return fig

# Callback to update the correlation matrix for cryptocurrencies
@app.callback(
    Output('crypto-correlation-matrix', 'figure'),
    [Input('crypto-dropdown', 'value')]
)
def update_crypto_correlation_matrix(selected_crypto):
    pivot_df = cryptocurrencies.pivot(index='timestamp', columns='crypto_name', values='price')
    correlation_matrix = pivot_df.corr()

    fig = px.imshow(correlation_matrix,
                    x=correlation_matrix.columns,
                    y=correlation_matrix.columns,
                    color_continuous_scale='RdBu_r',
                    zmin=-1, zmax=1)
    fig.update_layout(title='Correlation Matrix of Crypto Prices')
    
    return fig

# Callback to update daily returns graph for stocks
@app.callback(
    Output('daily-returns-graph', 'figure'),
    [Input('stock-dropdown', 'value')]
)
def update_daily_returns_graph(selected_stock):
    stock_data = stocks[stocks['stock_symbol'] == selected_stock]
    fig = px.line(stock_data, x='timestamp', y='daily_return', title=f'Daily Returns for {selected_stock}')
    fig.update_layout(yaxis_title='Daily Return', xaxis_title='Timestamp')
    return fig

# Callback to update daily returns graph for cryptocurrencies
@app.callback(
    Output('crypto-daily-returns-graph', 'figure'),
    [Input('crypto-dropdown', 'value')]
)
def update_crypto_daily_returns_graph(selected_crypto):
    crypto_data = cryptocurrencies[cryptocurrencies['crypto_name'] == selected_crypto]
    fig = px.line(crypto_data, x='timestamp', y='daily_return', title=f'Daily Returns for {selected_crypto}')
    fig.update_layout(yaxis_title='Daily Return', xaxis_title='Timestamp')
    return fig

# Callback to update cumulative returns graph for stocks
@app.callback(
    Output('cumulative-returns-graph', 'figure'),
    [Input('stock-dropdown', 'value')]
)
def update_cumulative_returns_graph(selected_stock):
    stock_data = stocks[stocks['stock_symbol'] == selected_stock]
    fig = px.line(stock_data, x='timestamp', y='cumulative_return', title=f'Cumulative Returns for {selected_stock}')
    fig.update_layout(yaxis_title='Cumulative Return', xaxis_title='Timestamp')
    return fig

# Callback to update cumulative returns graph for cryptocurrencies
@app.callback(
    Output('crypto-cumulative-returns-graph', 'figure'),
    [Input('crypto-dropdown', 'value')]
)
def update_crypto_cumulative_returns_graph(selected_crypto):
    crypto_data = cryptocurrencies[cryptocurrencies['crypto_name'] == selected_crypto]
    fig = px.line(crypto_data, x='timestamp', y='cumulative_return', title=f'Cumulative Returns for {selected_crypto}')
    fig.update_layout(yaxis_title='Cumulative Return', xaxis_title='Timestamp')
    return fig

# Combined callback to manage stock portfolio and display metrics
@app.callback(
    [Output('portfolio-display', 'children'),
     Output('portfolio-metrics', 'children'),
     Output('portfolio-remove-dropdown', 'options'),
     Output('stock-distribution', 'figure')],
    [Input('add-to-portfolio-button', 'n_clicks'),
     Input('remove-from-portfolio-button', 'n_clicks')],
    [State('portfolio-stock-input', 'value'),
     State('portfolio-date-picker', 'date'),
     State('portfolio-quantity-input', 'value'),
     State('portfolio-price-input', 'value'),
     State('portfolio-remove-dropdown', 'value')]
)
def manage_portfolio(add_clicks, remove_clicks, stock_symbol, purchase_date, quantity, price, remove_symbol):
    ctx = dash.callback_context
    if not ctx.triggered:
        portfolio_list = [f"{stock['symbol']} - {stock['quantity']} shares bought on {stock['date']} at ${stock['price']}" for stock in portfolio]
        
        total_value = 0
        total_quantity = 0
        total_investment = 0
        stock_metrics = []

        for stock in portfolio:
            stock_data = stocks[stocks['stock_symbol'] == stock['symbol']]
            if not stock_data.empty:
                current_price = stock_data['price'].iloc[-1]
                investment_value = stock['quantity'] * current_price
                total_investment += stock['quantity'] * stock['price']
                profit_loss = investment_value - (stock['quantity'] * stock['price'])
                total_value += investment_value
                total_quantity += stock['quantity']
                stock_metrics.append(f"{stock['symbol']}: {stock['quantity']} shares, Bought at ${stock['price']}, Current Price: ${current_price:.2f}, Investment Value: ${investment_value:.2f}, P/L: ${profit_loss:.2f}")

        portfolio_metrics = html.Ul([
            html.Li(f"Total Portfolio Value: ${total_value:.2f}"),
            html.Li(f"Total Shares Owned: {total_quantity}"),
            html.Li(f"Total Investment: ${total_investment:.2f}"),
            html.Li(f"Total Profit/Loss: ${total_value - total_investment:.2f}")
        ] + [html.Li(metric) for metric in stock_metrics])

        options = [{'label': stock['symbol'], 'value': stock['symbol']} for stock in portfolio]

        stock_distribution = px.pie(
            pd.DataFrame(portfolio),
            names='symbol',
            values='quantity',
            title='Stock Distribution'
        )

        return html.Ul([html.Li(item) for item in portfolio_list]), portfolio_metrics, options, stock_distribution

    button_id = ctx.triggered[0]['prop_id'].split('.')[0]

    if button_id == 'add-to-portfolio-button' and stock_symbol and purchase_date and quantity and price:
        new_entry = {
            'symbol': stock_symbol,
            'date': purchase_date,
            'quantity': quantity,
            'price': price
        }
        portfolio.append(new_entry)
    elif button_id == 'remove-from-portfolio-button' and remove_symbol:
        portfolio[:] = [stock for stock in portfolio if stock['symbol'] != remove_symbol]

    data['portfolio'] = portfolio
    write_data(DATA_FILE, data)
    
    portfolio_list = [f"{stock['symbol']} - {stock['quantity']} shares bought on {stock['date']} at ${stock['price']}" for stock in portfolio]
    
    total_value = 0
    total_quantity = 0
    total_investment = 0
    stock_metrics = []

    for stock in portfolio:
        stock_data = stocks[stocks['stock_symbol'] == stock['symbol']]
        if not stock_data.empty:
            current_price = stock_data['price'].iloc[-1]
            investment_value = stock['quantity'] * current_price
            total_investment += stock['quantity'] * stock['price']
            profit_loss = investment_value - (stock['quantity'] * stock['price'])
            total_value += investment_value
            total_quantity += stock['quantity']
            stock_metrics.append(f"{stock['symbol']}: {stock['quantity']} shares, Bought at ${stock['price']}, Current Price: ${current_price:.2f}, Investment Value: ${investment_value:.2f}, P/L: ${profit_loss:.2f}")

    portfolio_metrics = html.Ul([
        html.Li(f"Total Portfolio Value: ${total_value:.2f}"),
        html.Li(f"Total Shares Owned: {total_quantity}"),
        html.Li(f"Total Investment: ${total_investment:.2f}"),
        html.Li(f"Total Profit/Loss: ${total_value - total_investment:.2f}")
    ] + [html.Li(metric) for metric in stock_metrics])

    options = [{'label': stock['symbol'], 'value': stock['symbol']} for stock in portfolio]

    stock_distribution = px.pie(
        pd.DataFrame(portfolio),
        names='symbol',
        values='quantity',
        title='Stock Distribution'
    )

    return html.Ul([html.Li(item) for item in portfolio_list]), portfolio_metrics, options, stock_distribution

# Combined callback to manage crypto portfolio and display metrics
@app.callback(
    [Output('crypto-portfolio-display', 'children'),
     Output('crypto-portfolio-metrics', 'children'),
     Output('crypto-portfolio-remove-dropdown', 'options'),
     Output('crypto-distribution', 'figure')],
    [Input('add-to-crypto-portfolio-button', 'n_clicks'),
     Input('remove-from-crypto-portfolio-button', 'n_clicks')],
    [State('portfolio-crypto-input', 'value'),
     State('crypto-portfolio-date-picker', 'date'),
     State('crypto-portfolio-quantity-input', 'value'),
     State('crypto-portfolio-price-input', 'value'),
     State('crypto-portfolio-remove-dropdown', 'value')]
)
def manage_crypto_portfolio(add_clicks, remove_clicks, crypto_name, purchase_date, quantity, price, remove_crypto):
    ctx = dash.callback_context
    if not ctx.triggered:
        crypto_portfolio_list = [f"{crypto['symbol']} - {crypto['quantity']} units bought on {crypto['date']} at ${crypto['price']}" for crypto in crypto_portfolio]
        
        total_value = 0
        total_quantity = 0
        total_investment = 0
        crypto_metrics = []

        for crypto in crypto_portfolio:
            crypto_data = cryptocurrencies[cryptocurrencies['crypto_name'] == crypto['symbol']]
            if not crypto_data.empty:
                current_price = crypto_data['price'].iloc[-1]
                investment_value = crypto['quantity'] * current_price
                total_investment += crypto['quantity'] * crypto['price']
                profit_loss = investment_value - (crypto['quantity'] * crypto['price'])
                total_value += investment_value
                total_quantity += crypto['quantity']
                crypto_metrics.append(f"{crypto['symbol']}: {crypto['quantity']} units, Bought at ${crypto['price']}, Current Price: ${current_price:.2f}, Investment Value: ${investment_value:.2f}, P/L: ${profit_loss:.2f}")

        crypto_portfolio_metrics = html.Ul([
            html.Li(f"Total Crypto Portfolio Value: ${total_value:.2f}"),
            html.Li(f"Total Units Owned: {total_quantity}"),
            html.Li(f"Total Investment: ${total_investment:.2f}"),
            html.Li(f"Total Profit/Loss: ${total_value - total_investment:.2f}")
        ] + [html.Li(metric) for metric in crypto_metrics])

        crypto_options = [{'label': crypto['symbol'], 'value': crypto['symbol']} for crypto in crypto_portfolio]

        crypto_distribution = px.pie(
            pd.DataFrame(crypto_portfolio),
            names='symbol',
            values='quantity',
            title='Crypto Distribution'
        )

        return html.Ul([html.Li(item) for item in crypto_portfolio_list]), crypto_portfolio_metrics, crypto_options, crypto_distribution

    button_id = ctx.triggered[0]['prop_id'].split('.')[0]

    if button_id == 'add-to-crypto-portfolio-button' and crypto_name and purchase_date and quantity and price:
        new_entry = {
            'symbol': crypto_name,
            'date': purchase_date,
            'quantity': quantity,
            'price': price
        }
        crypto_portfolio.append(new_entry)
    elif button_id == 'remove-from-crypto-portfolio-button' and remove_crypto:
        crypto_portfolio[:] = [crypto for crypto in crypto_portfolio if crypto['symbol'] != remove_crypto]

    data['crypto_portfolio'] = crypto_portfolio
    write_data(DATA_FILE, data)
    
    crypto_portfolio_list = [f"{crypto['symbol']} - {crypto['quantity']} units bought on {crypto['date']} at ${crypto['price']}" for crypto in crypto_portfolio]
    
    total_value = 0
    total_quantity = 0
    total_investment = 0
    crypto_metrics = []

    for crypto in crypto_portfolio:
        crypto_data = cryptocurrencies[cryptocurrencies['crypto_name'] == crypto['symbol']]
        if not crypto_data.empty:
            current_price = crypto_data['price'].iloc[-1]
            investment_value = crypto['quantity'] * current_price
            total_investment += crypto['quantity'] * crypto['price']
            profit_loss = investment_value - (crypto['quantity'] * crypto['price'])
            total_value += investment_value
            total_quantity += crypto['quantity']
            crypto_metrics.append(f"{crypto['symbol']}: {crypto['quantity']} units, Bought at ${crypto['price']}, Current Price: ${current_price:.2f}, Investment Value: ${investment_value:.2f}, P/L: ${profit_loss:.2f}")

    crypto_portfolio_metrics = html.Ul([
        html.Li(f"Total Crypto Portfolio Value: ${total_value:.2f}"),
        html.Li(f"Total Units Owned: {total_quantity}"),
        html.Li(f"Total Investment: ${total_investment:.2f}"),
        html.Li(f"Total Profit/Loss: ${total_value - total_investment:.2f}")
    ] + [html.Li(metric) for metric in crypto_metrics])

    crypto_options = [{'label': crypto['symbol'], 'value': crypto['symbol']} for crypto in crypto_portfolio]

    crypto_distribution = px.pie(
        pd.DataFrame(crypto_portfolio),
        names='symbol',
        values='quantity',
        title='Crypto Distribution'
    )

    return html.Ul([html.Li(item) for item in crypto_portfolio_list]), crypto_portfolio_metrics, crypto_options, crypto_distribution

if __name__ == '__main__':
    app.run_server(debug=True)
   
